<a href="https://colab.research.google.com/github/Yoshani/Pump-It-Up/blob/master/Pump_It_Up_Best_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install category_encoders

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# classifiers
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier


# Pre processing

In [ ]:
features_df = pd.read_csv(
    "/content/drive/MyDrive/PumpItUp/TrainingSet.csv", 
    index_col="id",
    low_memory=False
)
labels_df = pd.read_csv(
    "/content/drive/MyDrive/PumpItUp/TrainingLabels.csv", 
    index_col="id"
)
test_df = pd.read_csv(
    "/content/drive/MyDrive/PumpItUp/TestSet.csv", 
    index_col="id",
    low_memory=False
)

In [ ]:
features_df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [ ]:
X = features_df.copy()
y = labels_df.copy()
X_test = test_df.copy()

In [ ]:
# Combine train data with test data
df=X.append(X_test)

In [ ]:
# Drop some columns
df.drop(['recorded_by', 'num_private', 'wpt_name', 'extraction_type_group', 'extraction_type', 'payment_type', 'water_quality', 'scheme_management',
         'district_code', 'region','region_code', 'subvillage', 'ward', 'waterpoint_type_group', 'quantity_group', 'installer', 'scheme_name'], axis=1, inplace=True)

In [ ]:
df.head()

,amount_tsh,date_recorded,funder,gps_height,longitude,latitude,basin,lga,population,public_meeting,permit,construction_year,extraction_type_class,management,management_group,payment,quality_group,quantity,source,source_type,source_class,waterpoint_type
id,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,34.938093,-9.856322,Lake Nyasa,Ludewa,109,True,False,1999,gravity,vwc,user-group,pay annually,good,enough,spring,spring,groundwater,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,34.698766,-2.147466,Lake Victoria,Serengeti,280,NaN,True,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,37.460664,-3.821329,Pangani,Simanjiro,250,True,True,2009,gravity,vwc,user-group,pay per bucket,good,enough,dam,dam,surface,communal standpipe multiple
67743,0.0,2013-01-28,Unicef,263,38.486161,-11.155298,Ruvuma / Southern Coast,Nanyumbu,58,True,True,1986,submersible,vwc,user-group,never pay,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple
19728,0.0,2011-07-13,Action In A,0,31.130847,-1.825359,Lake Victoria,Karagwe,0,True,True,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe


In [ ]:
# construction_year
med = df.loc[df['construction_year'] > 0,'construction_year'].median()
df.loc[df['construction_year'] == 0,'construction_year'] = med

min_year = 1960
df['construction_year'] = df['construction_year'] - min_year

# gps_height
med = df.loc[df['gps_height'] > 0,'gps_height'].median()
df.loc[df['gps_height'] == 0,'gps_height'] = med

# funder
df['funder'] = df['funder'].fillna("Unknown")

# other
df = df.fillna(df.median())

# label encode
for colname in df.select_dtypes(["object", "category"]):
        df[colname], _ = df[colname].factorize()

In [ ]:
# Number of missing values in each column
missing_val_count_by_column = (df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


Separate training and test set

In [ ]:
X = df[:len(features_df)]
X_test = df[len(features_df):]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y.status_group, test_size=0.1, random_state=10)

# Train model

Define models

In [ ]:
modelRFC = RandomForestClassifier(n_estimators=1300, min_samples_split=10, criterion='gini')

modelETC = ExtraTreesClassifier(n_estimators=1000, min_samples_split=10)

modelXGB = XGBClassifier(objective = 'multi:softmax', 
                      num_class = 3, maximize = False, eval_metric = 'merror', eta = 0.1,
                      max_depth = 14, colsample_bytree = 0.4, num_boost_round=1000, nfold=4, early_stopping_rounds=10)

modelGB = GradientBoostingClassifier(random_state=8, n_estimators=250)

modelLGB = lgb.LGBMClassifier(learning_rate=0.03, n_estimators=478, max_depth=5, min_samples_split=100, min_samples_leaf=100, random_state=27, gamma=0.1,
                                                       subsample=0.8, reg_alpha = 0.001, reg_lambda=10, objective = 'binary', boosting_type='goss')

modelSVC = SVC(gamma='auto', kernel='rbf')

modelCAT = CatBoostClassifier(silent=True, max_ctr_complexity=5, task_type='CPU', iterations=10000,
        eval_metric='AUC', od_type='Iter', od_wait=500, loss_function = 'MultiClass', random_seed=42)

modelMLP = MLPClassifier(hidden_layer_sizes=(200,100,), learning_rate='invscaling', learning_rate_init=0.01,
                          power_t=0.5, max_iter=1000, shuffle=True, tol=1e-4, activation='logistic', 
                          early_stopping=True, validation_fraction=0.1, n_iter_no_change=12, max_fun=15000,
                         random_state=8, solver='adam',batch_size=1024)

Single scores

In [ ]:
modelRFC.fit(X_train, y_train)
scoreRFC = modelRFC.score(X_valid,y_valid)
scoreRFC

0.8223905723905723

In [ ]:
modelETC.fit(X_train, y_train)
scoreETC = modelETC.score(X_valid,y_valid)
scoreETC

0.8171717171717172

In [ ]:
modelXGB.fit(X_train, y_train)
scoreXGB = modelXGB.score(X_valid,y_valid)
scoreXGB

0.8235690235690236

In [ ]:
modelCAT.fit(X_train, y_train)
scoreCAT = modelCAT.score(X_valid,y_valid)
scoreCAT

0.8028619528619528

Ensembling

In [ ]:
estimators_stacked = [
              ('xgb', modelXGB),
              ('rn', modelRFC)              
             ]

# Stacking Classifier
stacked_estimator =  StackingClassifier(estimators=estimators_stacked,
                                    final_estimator=LogisticRegression()  
                                  )
# Voting Classifier
voting_estimator = VotingClassifier(estimators=estimators_stacked, 
                        voting='hard')

In [ ]:
stacked_estimator.fit(X_train, y_train)
scoreStack = stacked_estimator.score(X_valid,y_valid)
scoreStack

0.8260942760942761

In [ ]:
voting_estimator.fit(X_train, y_train)
scoreVoting = voting_estimator.score(X_valid,y_valid)
scoreVoting

0.8217171717171717

# Train on entire dataset

In [ ]:
model=stacked_estimator
X_train=X
y_train = y.status_group

model.fit(X_train, y_train)

# Predict

In [ ]:
predict=model.predict(X_test)

# Submission

In [ ]:
submission_df = pd.read_csv("/content/drive/MyDrive/PumpItUp/SubmissionFormat.csv", 
                            index_col="id")

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["status_group"] = predict

In [ ]:
submission_df['status_group'].unique()

array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

In [ ]:
submission_df.to_csv('my_submission.csv', index=True)

In [ ]:
from google.colab import files
files.download('my_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>